In [34]:
import pandas as pd
import numpy as np
import torch
import os
import torch.nn as nn                  
import torch.nn.functional as F       
import torch.optim as optim            
import torch.utils.data as data
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchmetrics.functional as metrics 

In [35]:
irisDF = pd.read_csv('../data/iris.csv')
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [115]:
target = irisDF['variety']
feature = irisDF.drop('variety', axis = 1)

In [37]:
target.replace({'Setosa': 0, 'Versicolor': 1,'Virginica':2}, inplace=True)

In [44]:
class DS(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [45]:
irisDS = DS(feature, target)

In [46]:
# __getitem__ 메서드 소환
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [47]:
irisDS.target, irisDS.feature

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2]),
 tensor([[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [4.7000, 3.2000, 1.3000, 0.2000],
         [4.6000, 3.1000, 1.5000, 0.2000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.4000, 1.4000, 0.3000],
         [5.0000, 3.4000, 1.5000, 0.2000],
         [4.4000, 2.9000, 1.4000, 0.2000],
         [4.9000, 3.1000, 1.5000, 0.1000],
         [5.4000, 3.7000, 1.5000, 0.2000],
         [4.

#### DataLoder
- batch_size, shuffle, drop_last, sampler
- sampler :인덱스를 꺼내주는 역할
- sampler랑 shuffle이랑 동시에 사용 불가
    * shuffle = True -> 자동으로 randeom sampler 작동
    * if not -> sequential sampler 지정
    * shuffle은 에포크 단위

In [88]:
from torch.utils.data import WeightedRandomSampler

In [87]:
# 여기에 사용자 정의 샘플러를 작성하고 활용할 수 있습니다.

# class WeightedRandomSampler(Sampler[int]):
#     r"""Samples elements from ``[0,..,len(weights)-1]`` with given probabilities (weights).
# 
#     Args:
#         weights (sequence)   : a sequence of weights, not necessary summing up to one
#         num_samples (int): number of samples to draw
#         replacement (bool): if ``True``, samples are drawn with replacement.
#             If not, they are drawn without replacement, which means that when a
#             sample index is drawn for a row, it cannot be drawn again for that row.
#         generator (Generator): Generator used in sampling.
# 
#     Example:
#         >>> # xdoctest: +IGNORE_WANT("non-deterministic")
#         >>> list(WeightedRandomSampler([0.1, 0.9, 0.4, 0.7, 3.0, 0.6], 5, replacement=True))
#         [4, 4, 1, 4, 5]
#         >>> list(WeightedRandomSampler([0.9, 0.4, 0.05, 0.2, 0.3, 0.1], 5, replacement=False))
#         [0, 1, 4, 3, 2]
#     """
# 
#     weights: Tensor
#     num_samples: int
#     replacement: bool
# 
#     def __init__(self, weights: Sequence[float], num_samples: int,
#                  replacement: bool = True, generator=None) -> None:
#         if not isinstance(num_samples, int) or isinstance(num_samples, bool) or \
#                 num_samples <= 0:
#             raise ValueError(f"num_samples should be a positive integer value, but got num_samples={num_samples}")
#         if not isinstance(replacement, bool):
#             raise ValueError(f"replacement should be a boolean value, but got replacement={replacement}")
# 
#         weights_tensor = torch.as_tensor(weights, dtype=torch.double)
#         if len(weights_tensor.shape) != 1:
#             raise ValueError("weights should be a 1d sequence but given "
#                              f"weights have shape {tuple(weights_tensor.shape)}")
# 
#         self.weights = weights_tensor
#         self.num_samples = num_samples
#         self.replacement = replacement
#         self.generator = generator
# 
#     def __iter__(self) -> Iterator[int]:
#         rand_tensor = torch.multinomial(self.weights, self.num_samples, self.replacement, generator=self.generator)
#         yield from iter(rand_tensor.tolist())
# 
#     def __len__(self) -> int:
#         return self.num_samples


ModuleNotFoundError: No module named 'Sampler'

In [104]:
### 로딩된 데이터 확인 함수
def print_batch_data(loader, epochs,  batch_size=1, shuffle = False, drop_last =False, sampler = None):
    print(f'[설정값] batch size: {batch_size}, shuffle: {shuffle}, drop_last: {drop_last}, sampler: {sampler}')
    print(f'[loader] {loader}')
    
    for ep in range(epochs):
        print(f'[{ep+1} EPOCHS] ================ batch : {len(loader)}개')
        for (feature, target) in loader:  
            print(feature.shape, target.shape, target)

In [72]:
### ===> [기본] batch_size=1, shuffle = False, drop_last =False, sampler = None
loader = DataLoader(irisDS)

print_batch_data(loader = loader, epochs=2)

[설정값] batch size: 1, shuffle: False, drop_last: False, sampler: None
[loader] <torch.utils.data.dataloader.DataLoader object at 0x00000198390D3520>
[1 EPOCHS] ================ batch : 150개
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([1, 4]) torch.Size([1]) tensor([0])
torch.Size([

In [77]:
for (feature, target) in loader:
    print(target.bincount())  # 불균형한 데이터

tensor([8, 7, 7])
tensor([8, 7, 7])
tensor([5, 9, 8])
tensor([7, 6, 9])
tensor([6, 9, 7])
tensor([11,  6,  5])


In [74]:
### ===> [설정] batch_size=22, shuffle = False, drop_last =True, sampler = None
loader = DataLoader(irisDS,batch_size=22, drop_last=True)

print_batch_data(loader = loader, epochs=2)

[설정값] batch size: 1, shuffle: False, drop_last: False, sampler: None
[loader] <torch.utils.data.dataloader.DataLoader object at 0x00000198390D5250>
[1 EPOCHS] ================ batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
[2 EPOCHS] ================ batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [75]:
### ===> [설정] batch_size=22, shuffle = True, drop_last =True, sampler = None
loader = DataLoader(irisDS,batch_size=22, drop_last=True, shuffle=True)

print_batch_data(loader = loader, epochs=2)

[설정값] batch size: 1, shuffle: False, drop_last: False, sampler: None
[loader] <torch.utils.data.dataloader.DataLoader object at 0x00000198390F5700>
[1 EPOCHS] ================ batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([2, 1, 0, 2, 2, 2, 1, 2, 1, 0, 0, 2, 0, 1, 1, 2, 0, 2, 2, 1, 0, 2])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 1, 1, 2, 0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 2, 1, 1, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 1, 2, 1, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 2, 0, 0, 2, 1, 2, 0, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 2, 2, 0, 1, 1, 2, 2, 1, 2, 0, 1, 2, 1, 2, 1, 0, 0, 2, 1, 1, 1])
torch.Size([22, 4]) torch.Size([22]) tensor([2, 0, 2, 2, 0, 2, 2, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0])
torch.Size([22, 4]) torch.Size([22]) tensor([0, 2, 0, 1, 0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 1, 0, 0, 2, 0, 2, 1, 2])
[2 EPOCHS] ================ batch : 6개
torch.Size([22, 4]) torch.Size([22]) tensor([0, 1, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 0, 2, 1, 2, 

In [55]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 100
MODEL = model(feature.shape[1],target.nunique())
LF = nn.CrossEntropyLoss().to(DEVICE)
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr = 0.001)
SCADULER = ReduceLROnPlateau(OPTIMIZER, mode='min', patience=3)